In [1]:
from keras.layers import Input, Activation, Dense, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Masking, TimeDistributed, Lambda
from keras.models import Model
from keras.losses import binary_crossentropy
from keras.optimizers import SGD, Adam
import keras

Using TensorFlow backend.


In [2]:
from mask_utilities import ZeroMaskedEntries, mask_aware_max, mask_aware_mean, mask_aware_mean_output_shape

In [3]:
from sklearn.datasets import make_classification
import numpy as np

In [4]:
def create_mil_dataset(bag_sizes, prob_inst_positive=None, random_state=123456, **make_classification_kwargs):
    """This is not going to be efficient"""
    if prob_inst_positive is None:
        prob_inst_positive = 1 - np.exp(np.log(0.5)/np.mean(bag_sizes))
        
    X, y= make_classification(n_samples=np.asarray(bag_sizes).sum()*3, random_state=random_state)
    negative = X[y==0]
    positive = X[y==1]
    neg_pos = 0
    pos_pos = 0
    bags = []
    labels = []
    for i in range(len(bag_sizes)):
        bagdata = []
        baglabels = (np.random.uniform(size=bag_sizes[i]) > (1-prob_inst_positive)).astype(int)
        for lab in baglabels:
            if lab == 0:
                bagdata.append(negative[neg_pos])
                neg_pos += 1
            else:
                bagdata.append(positive[pos_pos])
                pos_pos += 1
        bagdata = np.array(bagdata)
#         print(bagdata.shape)
#         print(baglabels.shape)
        bags.append(np.hstack([bagdata, baglabels.reshape(-1, 1)]))
        labels.append(max(baglabels))
    bags = np.array(bags)
    labels = np.array(labels)
    return bags,labels

In [5]:
dataset, labels = create_mil_dataset(np.array([1,2,3,4]))
dataset

array([ array([[-0.02746227,  0.24064694,  0.34312528,  0.45242869,  2.13782675,
         0.48310266,  0.87783281, -0.93118368,  0.32410174, -1.23386202,
        -0.58196715, -0.65841844,  1.23327337, -3.52087607, -1.40304778,
         0.51686997,  0.37521779, -0.0614506 , -0.21051828,  0.03311313,
         1.        ]]),
       array([[ 2.12145332,  0.60460316,  0.37795332, -2.46146711,  0.49367226,
        -1.55390218,  0.79549483,  1.48734885,  0.96766064, -0.92879728,
         1.3750197 ,  0.56369984, -0.74320592, -0.30885258, -1.05790875,
        -0.68108726, -1.4331399 ,  0.87556247,  1.14599999,  0.59770053,
         1.        ],
       [-0.31744079,  3.35742705,  1.51997029,  0.60017759, -0.49366211,
         0.27422987, -0.92999096,  0.01487095, -0.48760159,  0.38039557,
        -2.18293672,  0.89617065, -1.96978545,  0.08484421, -0.08224033,
         0.43238977, -1.02951028,  0.39702147,  2.39678042, -1.23626892,
         0.        ]]),
       array([[ 0.44115327, -1.07135667

In [8]:
np.mean(create_mil_dataset(np.array([6]*200))[1])  # balancing of classes is working

0.47999999999999998

In [85]:
dataset, labels = create_mil_dataset(np.random.randint(5, 20, size=600))
labels

array([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 1,

In [86]:
def extract_sil_dataset(mil_dataset):
    sil_dataset = None
    for bag in dataset:
        if sil_dataset is None:
            sil_dataset = bag
        sil_dataset = np.vstack([sil_dataset, bag])
    return sil_dataset[:, :-1], sil_dataset[:, -1]

In [87]:
X_sil, Y_sil = extract_sil_dataset(dataset)
X_sil.shape

(7249, 20)

In [88]:
nfeats = dataset[0][0].shape[0] - 1

In [144]:
inlayer = Input([None, nfeats])
masked_input = Masking()(inlayer)
layer2 = TimeDistributed(Dense(30, activation='relu'))(masked_input)
layer3 = TimeDistributed(Dense(1, activation='relu'))(layer2)
instance_predictions = Activation('sigmoid')(layer3)
zeroed_layer3 = ZeroMaskedEntries()(instance_predictions)
layer4 = Lambda(mask_aware_max, mask_aware_mean_output_shape)(zeroed_layer3)
model = Model([inlayer], [layer4])

In [136]:
[l.output_shape for l in model.layers]

[(None, None, 20),
 (None, None, 20),
 (None, None, 30),
 (None, None, 1),
 (None, None, 1),
 (None, None, 1),
 (None, 1)]

In [137]:
[l for l in model.layers]

In [138]:
model.predict(np.array([dataset[0][:,:-1]]))

array([[ 0.83121371]], dtype=float32)

In [139]:
model.compile('rmsprop', 'mse', metrics=['accuracy'])

In [140]:
X = keras.preprocessing.sequence.pad_sequences([d[:, :-1] for d in dataset], dtype='float32', value=0)

In [141]:
X_train = X[:len(X)*3//4]
Y_train = labels[:len(X)*3//4]
X_test = X[len(X)*3//4:]
Y_test = labels[len(X)*3//4:]

In [142]:
cb = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=100)]
# cb = []

In [143]:
model.fit(X_train, Y_train, batch_size=32, epochs=1000, validation_data=(X_test, Y_test), callbacks=cb, verbose=2)

Train on 450 samples, validate on 150 samples
Epoch 1/1000
0s - loss: 0.3165 - acc: 0.4578 - val_loss: 0.2759 - val_acc: 0.4867
Epoch 2/1000
0s - loss: 0.2785 - acc: 0.4622 - val_loss: 0.2555 - val_acc: 0.5333
Epoch 3/1000
0s - loss: 0.2597 - acc: 0.4889 - val_loss: 0.2477 - val_acc: 0.5733
Epoch 4/1000
0s - loss: 0.2507 - acc: 0.5711 - val_loss: 0.2474 - val_acc: 0.5600
Epoch 5/1000
0s - loss: 0.2469 - acc: 0.5822 - val_loss: 0.2467 - val_acc: 0.5533
Epoch 6/1000
0s - loss: 0.2442 - acc: 0.6022 - val_loss: 0.2455 - val_acc: 0.5800
Epoch 7/1000
0s - loss: 0.2423 - acc: 0.6333 - val_loss: 0.2451 - val_acc: 0.5667
Epoch 8/1000
0s - loss: 0.2403 - acc: 0.6267 - val_loss: 0.2445 - val_acc: 0.5667
Epoch 9/1000
0s - loss: 0.2388 - acc: 0.6244 - val_loss: 0.2432 - val_acc: 0.5867
Epoch 10/1000
0s - loss: 0.2379 - acc: 0.6533 - val_loss: 0.2432 - val_acc: 0.5800
Epoch 11/1000
0s - loss: 0.2370 - acc: 0.6400 - val_loss: 0.2432 - val_acc: 0.5467
Epoch 12/1000
0s - loss: 0.2359 - acc: 0.6267 - va

In [ ]:
model.evaluate()